In [ ]:
import sys
dir1 = '/Users/erihe/OneDrive - NTNU/'
if not dir1 in sys.path: sys.path.append(dir1)
from utils_new import *

In [ ]:
ind_fis = {}
ind_fis['npI1_0417'] = ((1,(0,1,2)),
                        (4,(0,1,2)),
                        (5,(0,1,2)),)

ind_fis['npG2_1211'] = ((4,(0,)),)
ind_fis['npH1_0314'] = ((1,(1,)),)
ind_fis['npH3_0402'] = ((1,(0,1)),)
ind_fis['npI1_0414'] = ((0,(0,)), 
                         (1,(0,1)),)
ind_fis['npI1_0416'] = ((1,(1,)),)
ind_fis['npI5_0414'] = ((0,(1,)),
                         (3,(0,)))

ind_fis['npI5_0415'] = ((0,(2,)),)

ind_fis['npJ1_0521'] = ((2,(1,)),)
ind_fis['npJ1_0524'] = ((3,(1,2)),)
ind_fis['npJ5_0505'] = ((0,(0,1,2)),)
ind_fis['npJ5_0506'] = ((2,(0,1)),
                        (3,(0,)),
                        (4,(0,1)),
                        (5,(0,)),
                       )
ind_fis['npJ5_0507'] = ((0,(0,1)),)


# Compute and Plot

In [ ]:
data_dir = '/Users/erihe/OneDrive - NTNU/Giocomo/giocomo_data'
#os.mkdir('Results')
count1 = 0

k = 1000          # number of neighbours for downsampling
dt_curr = 0.01    # time bin interval
time_resolution = 100000 
sigma = 10
thr = 0.46
sp = 10
dim = 7
eps = 0.7
nn = 1
n_points = 1200
thr  = 0.46
sigma = 10
sp = 10
dim = 7
thrstr = str(int(thr*100))    
maxdim = 1
dt_curr = 0.01
time_resolution = 100000
min_time = 0

for mouse_sess in ind_fis:# 'npJ5_0506'
    mouse_folder = 'Results/' + mouse_sess
#    os.mkdir(mouse_folder)

    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    print(files)

    
    ################ load data ######################
    cells_inds = {}
    indskeep_all = {}
    offsets = {}
    xcoords = {}
    ycoords = {}
    vr_session_offset = {}
    spike_depth = {}
    depth = {}
    regions = {}
    for fi in files:
        data = loadmat(fi)
        if 'anatomy' in list(data.keys()):
            anatomy = data['anatomy']
            if 'parent_shifted' in anatomy:
                group = anatomy['parent_shifted']
            else:
                group = anatomy['cluster_parent']
            depth[fi] = data['anatomy']['tip_distance']
        else:
            group = np.array(['MEC' for n in range(len(data['sp']['cids']))])
            depth[fi] = np.array([0 for n in range(len(data['sp']['cids']))])
        cells_inds[fi] = data['sp']['cids']
        regions[fi] = group
        xcoords[fi] = data['sp']['xcoords']
        ycoords[fi] = data['sp']['ycoords']
        indskeep_all[fi]  = data['sp']['cgs']

    data_trials = {}
    data_pos = {}
    sspk1 = {}
    spk_count1 = {}
    speed = {}
    posx = {}
    post = {}
    posxx = {}
    postt = {}
    postrial = {}
    gain = {}
    contrast = {}
    lickt = {}
    lickx = {}
    spike_times = {}

    max_times = {}
    for fi in files[:]:
        data = loadmat(fi)
        spikes = {}
        for cell_idx in range(len(cells_inds[fi])):   
            spikes[cell_idx] = data['sp']['st'][data['sp']['clu'] == cells_inds[fi][cell_idx]]    
        spike_times[fi] = spikes.copy()

        max_time = data['sp']['st'].max()+dt_curr
        max_times[fi] = max_time.copy()
        spk_count1[fi], tt = spk_count(spikes, min_time = min_time, max_time = max_time, dt_orig = dt_curr, res = time_resolution)
        sspk1[fi], __ = firing_rate(spikes, sigma = sigma/10, min_time = min_time, max_time = max_time, 
                                 dt_orig = dt_curr*10, res = time_resolution/10)

        data = loadmat(fi)

        max_time = data['sp']['st'].max() +dt_curr
        max_times[fi] = max_time.copy()

        xtmp = data['posx'].copy()
        xtmp += (data['trial']-1)*400


        postt[fi], posxx[fi], __, speed[fi],__ = get_pos(xtmp, [], data['post'], [],
                                                         dt_curr, res = time_resolution, min_time = min_time, max_time = max_time)
        posxx[fi] = posxx[fi]%400

        postrial[fi] = data['trial']
        gain[fi] = data['trial_gain']
        contrast[fi] = data['trial_contrast']
        lickt[fi] = data['lickt']        
        lickx[fi] = data['lickx']
        tt_dig = np.digitize(postt[fi], data['post'])-1    
        postrial[fi] = data['trial'][tt_dig]
        data_trials[fi] = np.unique(data['trial'])


    indskeep = np.ones(len(indskeep_all[files[0]]), dtype = bool)
    for ii in indskeep_all:
        if len(indskeep) != len(indskeep_all[ii]):
            continue
        indskeep[~indskeep | ~(indskeep_all[ii]==2)] = False  
    indsreg = np.ones(len(regions[files[0]]), dtype = bool)
    for ii in regions:
        if len(indsreg) != len(regions[ii]):
            continue
        indsreg[~indsreg | ~(regions[ii]=='MEC')] = False  

    num_neurons = len(spk_count1[files[0]][0,:])
    indsnull = np.ones(num_neurons,dtype =bool)
    meanRate = {}
    for fi in spk_count1:
        spksum = np.sum(spk_count1[fi], 0)/len(spk_count1[fi])*100
        indsnull[(spksum<0.05) | (spksum>10)] = False
        meanRate[fi] = spksum.copy()

    ############# compute temporal cross- and autocorrelagroms #############
    if len(glob.glob(data_dir + '/' + mouse_sess + '_corrs.npz')) ==0:
        num_neurons = len(sspk1[files[0]][0,:])
        Xcorr = {}
        for fi in files:
            sspikes = np.sqrt(sspk1[fi])
            t0 = time.time()
            Xcorr[fi] =  cross_corr_dist(sspikes, lencorr = 30)
        acorrs = get_acorr(mouse_sess, good_cells = np.arange(num_neurons), data_trials = data_trials,
                          sspk1 = spk_count1, files = files, posxx = posxx, postrial = postrial)
        t_acorrs = {}
        for fi in files:
            print(fi)
            t_acorr = get_temporal_acorr(spike_times[fi], maxspikes = 1000, inds = np.where(indsnull)[0])
            t_acorr = t_acorr[:, 200:]
            t_acorr = t_acorr.astype(float)
            for i in range(len(t_acorr[:,0])):
                t_acorr[i,:] = t_acorr[i,:]/t_acorr[i,0]
            t_acorr[:,0] = 0
            t_acorr = gaussian_filter1d(t_acorr[:, :],sigma = 2, axis = 1)
            t_acorrs[fi] = t_acorr.copy()
        np.savez(data_dir + '/' + mouse_sess + '_corrs', 
                 acorr = acorrs, 
                 t_acorr = t_acorrs, 
                 Xcorr = Xcorr)
    else:
        f = np.load(data_dir + '/' + mouse_sess + '_corrs.npz', allow_pickle = True) 
        acorrs = f['acorr']
        t_acorrs = f['t_acorr']
        Xcorr = f['Xcorr']
        f.close()

    ############# Compute distance matrix and cluster ensembles  #############
    num_neurons = len(Xcorr[files[0]])
    indskeep_bu = np.where(indsnull & indskeep )[0]#np.arange(num_neurons)
    d1 = np.zeros((len(indskeep_bu),len(indskeep_bu)))
    for x in Xcorr:    
        xtmp = Xcorr[x]
        xtmp = xtmp[indskeep_bu,:]
        xtmp = xtmp[:, indskeep_bu]
        dd1 = squareform(pdist(np.square(xtmp), 'correlation'))
        d1 += dd1
    infs = np.isinf(d1)
    d1[infs] = np.max(d1[~infs])*2
    d1 /= len(Xcorr)
    dvals = d1[np.triu_indices(len(d1),1)]

    ind1 = get_ind(d1,thr, linkage = 'average', bPlot = False)
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0] 

    np.savez('Results/' + mouse_sess + '/inds', 
             ind1 = ind1, 
             indskeep_bu = indskeep_bu, 
             indsnull = indsnull,
             indskeep = indskeep,
             indsreg = indsreg                
            )

    ############# Perform topological analyses and determine toroidal score  #############
    dgms_all = {}
    coords_ds_all = {}
    pcorrs_hex = {}
    pcorrs_sqr = {}
    curr_ind_fis = ind_fis[mouse_sess]
    for indit in curr_ind_fis:
        ii = rel_inds[indit[0]]
        for fiit in indit[1]:            
            fi = files[fiit]
            currsess = fi[fi.find(mouse_sess)+len(mouse_sess)+1:-4]
            mod_ind1s = indskeep_bu[np.where(ind1==ii)[0]]
            spktimes_tmp = {}
            count = 0
            for i, spk in enumerate(spike_times[fi]):
                if i in mod_ind1s:
                    spktimes_tmp[count] = spike_times[fi][spk]
                    count += 1
            sspikes_of1 = firing_rate(spktimes_tmp, sigma = sigma, min_time = 0, max_time = max_times[fi], 
                                      dt_orig = dt_curr, res = time_resolution)[0]
            sspikes_of1 = np.sqrt(sspikes_of1)        
            movetimes0 = np.arange(0,len(sspikes_of1), nn) 
            movetimes0 = movetimes0[np.where(speed[fi][movetimes0]>sp)[0]]
            spk1 = preprocessing.scale(sspikes_of1[movetimes0,:],axis = 0)
            dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(spk1, dim = dim)
            dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])                        

            startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
            movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, epsilon = eps, startindex = startindex)
            indstemp = []
            n_points = min(n_points, len(movetimes1))
            indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                               n_points, 1, metric = 'cosine')[0]
            indstemp = movetimes1[indstemp]
            dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]
            X = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
            thresh = np.max(X[~np.isinf(X)])    
            persistence = ripser(X, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
            coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], 
                bConsistent = True)

            coords_ds_all[str(indit) + '_' + str(fiit)] = coords_ds_consistent.copy()
            dgms_all[str(indit) + '_' + str(fiit)] = persistence['dgms'].copy()

            coords_mod2 = coords_ds_consistent.T*2*np.pi
            spk = sspikes_of1[movetimes0[indstemp],:]

            inds, inds_label =  get_coord_distribution(coords_mod2, 
                numbins = 50,epsilon = 0.1, metric = 'euclidean', startindex = -1)   
            phases_1 = get_phases(spk, coords_mod2, inds, inds_label)  
            pcorr1 = match_phases(coords_mod2, spk, phases_1)

            coords_mod1 = coords_mod2.copy()
            coords_mod1[:,0] = 2*np.pi - coords_mod1[:,0]

            phases_2 = get_phases(spk, coords_mod1, inds, inds_label)  
            pcorr2 = match_phases(coords_mod1, spk, phases_2)
            if np.median(pcorr2)> np.median(pcorr1):
                pcorr1 = pcorr2
            pcorrsqr = match_phases(coords_mod2, spk, phases_1, bSqr = True)
            pcorrs_hex[str(indit) + '_' + str(fiit)] = pcorr1.copy()
            pcorrs_sqr[str(indit) + '_' + str(fiit)] = pcorrsqr.copy()

            dgms = persistence['dgms']
            plt.figure()

            plot_diagrams(dgms, list(np.arange(maxdim+1)), lifetime = True)
            plt.show()
            plot_barcode(dgms)

            fig, ax = plt.subplots(1,4, figsize = (10,5), dpi = 120)
            for i in range(len(coords_ds)):
                ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])

            ax[3].scatter(*coords_ds[:2,:], s = 100)
            for i in range(4):
                ax[i].set_aspect(1/ax[i].get_data_ratio())


            fig, ax = plt.subplots(1,4, figsize = (10,5), dpi = 120)
            for i in range(len(coords_ds)):
                ax[i].plot(coords_ds_consistent[i,np.argsort(coords_ds_consistent[i,:])])
            ax[3].scatter(*coords_ds_consistent[:2,:], s = 100)
            for i in range(4):
                ax[i].set_aspect(1/ax[i].get_data_ratio())
            plt.show()

            mod_name = 'mod' + str(ii)    
            dgms1 = {}
            for dind, d  in enumerate(dgms):
                dgms1[dind] = d

            fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess,'')
            np.savez(mouse_folder + '/' + mod_name + '_' + fi_name, 
                    sspikes_of1 = sspikes_of1, 
                    movetimes0 = movetimes0,
                    movetimes1 = movetimes1,
                    indstemp = indstemp,
                    coords_ds = coords_ds_consistent,
                    dgms = dgms1,
                    pcorr1 = pcorr1,
                    pcorr2 = pcorr2,
                    pcorrsqr = pcorrsqr 
                   )
    np.savez(mouse_folder + '/pcorrs_all', pcorrs_hex = pcorrs_hex, pcorrs_sqr = pcorrs_sqr)
    np.savez(mouse_folder + '/pcorrs_all', pcorrs_hex = pcorrs_hex, pcorrs_sqr = pcorrs_sqr)

## Toroidal score

In [ ]:
pcorrs_all_hex = {}
pcorrs_all_sqr = {}
countfin = 0
sig = 10
for mouse_sess in glob.glob('Results/np*'):
    msess = mouse_sess[mouse_sess.find('np'):]
    thrstr = str(int(thr*100))
    ff = glob.glob(mouse_sess + '/pcorrs_all.npz')
    f = np.load(ff[0], allow_pickle = True)
    phex = f['pcorrs_hex'][()]
    psqr = f['pcorrs_sqr'][()]
    f.close()
    pcorrs_all_hex[msess] = phex
    pcorrs_all_sqr[msess] = psqr


In [ ]:
pplot_hex = []
pplot_sqr = []
mall = []
for mouse_sess in pcorrs_all_hex:
    if len(pcorrs_all_hex[mouse_sess])>0:
        for p in pcorrs_all_hex[mouse_sess]:
            pplot_hex.append(pcorrs_all_hex[mouse_sess][p])
            pplot_sqr.append(pcorrs_all_sqr[mouse_sess][p])
            mall.append([mouse_sess])

num_mods = len(pcorrs_all_hex)
bSems = False
cs = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

fig = plt.figure(figsize=(2,6), dpi = 120)
ax = fig.add_subplot(111)
xs = [0.01, 0.09]
    
score_thr = 0.6
for ii in range(len(pplot_hex)):
    phex = pplot_hex[ii]
    psqr = pplot_sqr[ii]
    p1 = np.median(phex)
    p2 = np.median(psqr)
    p11 = np.std(phex)/np.sqrt(len(phex))
    p22 = np.std(psqr)/np.sqrt(len(psqr))
    if p1 > score_thr:
        print(mall[ii])
        if p2>p1:
            print(p1, p2)
        currcol = 'r'
    else:
        currcol = 'k'
    ax.scatter(xs, [p1, p2], 
               s = 150, marker='.', zorder=-1, 
               alpha  = 0.5, c = currcol)
    ax.plot(xs, [p1, p2], 
            lw = 1, alpha = 0.5, zorder = -2, c = currcol)
    ax.plot([xs[0],xs[0]],[p1-p11, p1+p11],  
            lw = 1, alpha = 0.5, zorder = -2, c = currcol)
    ax.plot([xs[1],xs[1]],[p2-p22, p2+p22],  
            lw = 1, alpha = 0.5, zorder = -2, c = currcol)

ax.set_xlim([0,0.1])
ax.set_ylim([0.,1])

ax.set_xticks(xs)
ys = [0,1]
#ys = [0, 75/360*2*np.pi, 150/360*2*np.pi]
ax.set_xticklabels(np.zeros(len(xs),dtype=str))
ax.xaxis.set_tick_params(width=1, length =7)
ax.set_yticks(ys)
ax.set_yticklabels(np.zeros(len(ys),dtype=str))
ax.yaxis.set_tick_params(width=1, length =7)


x0,x1 = ax.get_xlim()
y0,y1 = ax.get_ylim()
ax.set_aspect(abs(x1-x0)/(abs(y1-y0))*3.5)

plt.gca().axes.spines['top'].set_visible(False)
plt.gca().axes.spines['right'].set_visible(False)



# VR Acorr

In [ ]:
for fi in files:
    acorrs[fi] -= np.mean(acorrs[fi],1)[:,np.newaxis]
for i in np.unique(ind1):
    if sum(ind1==i)>10:
        print(sum(ind1==i))
        fig, ax = plt.subplots(1,1)
        aa = acorrs[files[0]][indskeep_bu][ind1==i]
        ax.imshow(acorrs[files[0]][indskeep_bu][ind1==i], vmin = 0, vmax = 0.1)
        ax.set_aspect(1/ax.get_data_ratio())
    

Distance tuning

In [ ]:
dist_name = "/Users/erihe/OneDrive - NTNU/Giocomo/GiocomoLab-Campbell_Attinger_CellReports-d825378/intermediate_data/dark_distance/dist_tun.mat"
dist_tuning = sio.loadmat(dist_name)
names = np.array(['Mouse', 'Date', 'MouseDate', 'Session', 'CellID', 'UniqueID', 'BrainRegion', 'InsertionDepth', 'DepthOnProbe',
         'DepthFromSurface', 'peak', 'period', 'prom', 'pval', 'peak_shuf', 'DepthAdjusted'])
dist_tun = {}
for i, names in enumerate(names):
    dist_tun[names] = dist_tuning['b']['data'][0,0][0,i]


In [ ]:
pval_cutoff = 0.01
min_prom = 0.1


In [ ]:
fi = 'npI1_0417_dark_1.mat'
cIdx_dist = np.where((dist_tun['Session'] == fi[:-4]) & (dist_tun['BrainRegion']=='MEC'))[0]
ids = (dist_tun['pval'][cIdx_dist]<pval_cutoff) & (dist_tun['prom'][cIdx_dist]>min_prom)


In [ ]:
acorrs0 = acorrs[data_dir + '/npI1_0417_dark_1.mat'][indskeep & indsreg].copy()
for i in range(len(acorrs0)):
    acorrs0[i, :] -= np.mean(acorrs0[i, :])
plt.imshow(acorrs0[ids[:,0],:], extent = [0,1,0,1], vmin = 0, vmax =0.1)


In [ ]:
acorrs0 = acorrs[data_dir + '/npI1_0417_dark_1.mat'][indskeep ].copy()
for i in range(len(acorrs0)):
    acorrs0[i, :] -= np.mean(acorrs0[i, :])
plt.imshow(acorrs0[np.argsort(ind1),:], extent = [0,1,0,1], vmin = 0, vmax =0.1)


In [ ]:
distr_all = {}
for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
    print(mouse_sess)
    f = np.load( data_dir + '/' + mouse_sess + '_scores.npz', allow_pickle = True)
    acorrs = f['acorr'][()]
    f.close()
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    indsnull = f['indsnull'] 
    indskeep = f['indskeep']
    f.close()
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
    for ii , fi in enumerate(files):
        files[ii] = fi.replace('\\', '/')              
        
    scores = np.zeros((len(ind_fi)+1,2))
    scores[0,0] = len(ind1)
    count = 0
    for ii, fis in ind_fi:
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        for fi in files:
            if fi.find('dark')>0:
                print(mod_name, fi)
                fi1 = fi.replace('giocomo_data/','').replace('.mat', '')
                cIdx_dist = np.where((dist_tun['Session'] == fi1))[0]
                ids = (dist_tun['pval'][cIdx_dist]<pval_cutoff) & (dist_tun['prom'][cIdx_dist]>min_prom)
                if len(ids)>0:
                    if count == 0:
                        scores[0,1] = sum(ids[indsnull[indskeep]])
                        count += 1
                    scores[count,0] = sum(ind1==ii)
                    scores[count,1] = sum(ids[indsnull[indskeep]][ind1==ii])
                    count += 1
    if count == 0:
        for ii, fis in ind_fi:
            count += 1
            ii = rel_inds[ii]
            scores[count,0] = sum(ind1==ii)         
    distr_all[mouse_sess] = scores
        

In [ ]:
plt.figure()
cs = ['#1f77b4', '#ff7f0e', '#2ca02c']
k = -1
for j, mouse_sess in enumerate(distr_all):    
    scores = distr_all[mouse_sess]
    print(mouse_sess)
    print(scores)
    print('')
    for i in range(1,len(scores)):
        plt.bar(np.arange(k,k+1), scores[i,1],width = 0.4, 
                bottom = 0,
            edgecolor = [[0.,0.,0.]],  lw = 0.5, color = [cs[1]])
        plt.bar(np.arange(k,k+1), scores[i,0] - scores[i,1],width = 0.4, 
                bottom = scores[i,1],
            edgecolor = [[0.,0.,0.]],  lw = 0.5, color = [cs[0]])
        k += 1
    plt.bar(np.arange(k,k+1), scores[0,1],width = 0.4, 
            bottom = 0,
        edgecolor = [[0.,0.,0.]],  lw = 0.5, color = [cs[1]])
    plt.bar(np.arange(k,k+1), scores[0,0] - scores[0,1],width = 0.4, 
            bottom = scores[0,1],
        edgecolor = [[0.,0.,0.]],  lw = 0.5, color = [cs[0]])
    k += 2
    

plt.xlim([-2,k-1])
plt.ylim([0,320])
plt.xticks([],[])
plt.yticks([],[])
plt.yticks([0,100,200, 300], ['' for i in [0,100,200, 300]])
plt.gca().axes.spines['top'].set_visible(False)
plt.gca().axes.spines['right'].set_visible(False)


## Plot Ratemaps

In [ ]:
for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
    dim = 7
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    f.close()
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
    for ii , fi in enumerate(files):
        files[ii] = fi.replace('\\', '/')

    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]

    for ii, fis in ind_fi:
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        coords_aligned = {}
        sspikes_of1_fis = {}
        movetimes0_fis = {}
        movetimes1_fis = {}
        indstemp_fis = {}
        coords_ds_fis = {}
        dgms_fis = {}
        theta_fis = {}
        for fi in fis:
            fi = files[fi]
            fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
            f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
            sspikes_of1_fis[fi] = f['sspikes_of1']
            movetimes0_fis[fi] = f['movetimes0']
            movetimes1_fis[fi] = f['movetimes1']
            indstemp_fis[fi] = f['indstemp']
            coords_ds_fis[fi] = f['coords_ds']
            pcorr1 = f['pcorr1']
            pcorr2 = f['pcorr2']
            f.close()
            if np.median(pcorr1)>np.median(pcorr2):
                theta_fis[fi] = True
            else:
                theta_fis[fi] = False
            coords_aligned[fi] = get_coords_all(sspikes_of1_fis[fi], 
                                          coords_ds_fis[fi],   
                                          movetimes0_fis[fi], 
                                          indstemp_fis[fi], 
                                          dim = 7, 
                                          bPCA = True,
                                         )

        fi1 = files[fis[0]]  
        if not theta_fis[fi1]:
            ctmp = coords_aligned[fi1].copy()
            ctmp[:,0] = 2*np.pi - ctmp[:,0]
            coords_aligned[fi1] = ctmp.copy()
            
            ctmp = coords_ds_fis[fi1].copy()
            ctmp[0,:] = 1 - ctmp[0,:]
            coords_ds_fis[fi1] = ctmp.copy()
            
            theta_fis[fi1] = True
        combs = {}
        
        pshift = {}
        if len(fis)>0:
            for fi in fis[1:]:
                fi = files[fi]
                coords_aligned[fi], combs[fi], pshift[fi] = align_coords(coords_aligned[fi][:,:2],
                                                                         movetimes0_fis[fi], 
                                                                         movetimes0_fis[fi1], 
                                                                         indstemp_fis[fi1], 
                                                                         coords_ds_fis[fi1],
                                                                         sspikes_of1_fis[fi], 
                                                                         sspikes_of1_fis[fi1]                                                             
                                                                        )
                print('')
        np.savez('Results/' + mouse_sess + '/' + mod_name + '_coords_aligned' , 
                 coords_aligned = coords_aligned, combs = combs, pshift = pshift, theta_fis = theta_fis)            


In [ ]:
############### Ratemaps ###############


for mouse_sess in ind_fis:
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
    for i, fi in enumerate(files):
        files[i] = fi.replace('\\', '/')

    ind_fi = ind_fis[mouse_sess]
    f = np.load(data_dir + '/' + mouse_sess + '_corrs.npz', allow_pickle = True)
    acorrs = f['acorr'][()]
    f.close()
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    f.close()
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]
    for ii, fis in ind_fi:
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        sspikes_of1_fis = {}
        pcorrmed = 0
        pcorr = []
        f = np.load('Results/' + mouse_sess + '/' + mod_name + '_coords_aligned.npz' , allow_pickle = True) 
        coords_aligned = f['coords_aligned'][()]
        f.close()
        
        for fi in fis:
            fi = files[fi]
            fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
            f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
            sspikes_of1_fis[fi] = f['sspikes_of1']
            pcorr1 = f['pcorr1']
            pcorr2 = f['pcorr2']
            f.close()
            if np.median(pcorr1)> pcorrmed:
                pcorrmed = np.median(pcorr1)
                pcorr = pcorr1
            if np.median(pcorr2)> pcorrmed:
                pcorrmed = np.median(pcorr2)
                pcorr = pcorr1            
                        
                        

        num_neurons = 4#len(pcorr)
        numfigs = len(fis) + 1
        numw = 1
        numh = int(np.ceil(num_neurons/numw))
        outer1 = gridspec.GridSpec(1, numw)
        fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 120)
        plt.viridis()
        nw = 0
        sig1 = 1
        numbins1 = 30
                        
        torsort = np.flip(np.argsort(pcorr))[:num_neurons]
        fi_dark = ''
        for fi in files:
            if fi.lower().find('dark')>-1:
                fi_dark = fi
                break
        if len(fi_dark) == 0:
            fi_dark = files[0]
                        
        ac = acorrs[fi_dark][indskeep_bu[ind1==ii],:].copy()

        for nn, n in enumerate(torsort):
            nnn = nn%numh
            if nnn == 0:
                outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .3)
                gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.2,wspace = .0)
                nw += 1
            xnum = 0

            ax = plt.subplot(gs2[nnn,xnum]) 
            xnum += 1
            ax.bar(np.arange(300),ac[n,:300], width = 1, color = 'k')
            ax.set_xlim([0,300])
            ax.set_ylim([0, 0.3])
            ax.set_xticks([0, 100, 200, 300])
            ax.set_yticks([0.0, 0.3,])
            ax.set_yticklabels('')
            ax.set_xticklabels('')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.set_aspect(1/ax.get_data_ratio())
            ax.set_xticks([])
            ax.set_yticks([])

            for fi in fis:
                fi = files[fi]
                ax = plt.subplot(gs2[nnn,xnum]) 
                xnum += 1    
                mtot_tmp, __, __, circ  = binned_statistic_2d(coords_aligned[fi][:,0],
                                                          coords_aligned[fi][:,1],
                                                          sspikes_of1_fis[fi][:,n], 
                                                          statistic = 'mean', 
                                                          bins = 30,
                                                          expand_binnumbers = True)    
                nans = np.isnan(mtot_tmp)
                mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])                
                mtot_tmp = smooth_tuning_map(mtot_tmp, numbins1+1, sig1, bClose = False) 
                mtot_tmp[nans] = -np.inf
                maxtot = np.sort(mtot_tmp.flatten())
                mintot = maxtot[int(0.025*len(maxtot))]
                maxtot = maxtot[int(0.975*len(maxtot))]
                        
                ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], vmin = 0, vmax = maxtot)
                ax.set_xticks([])
                ax.set_yticks([])
                ax.set_aspect(1/ax.get_data_ratio())
                r_box = transforms.Affine2D().skew_deg(15,15)
                for x in ax.images + ax.lines + ax.collections:
                    trans = x.get_transform()
                    x.set_transform(r_box+trans) 
                    if isinstance(x, PathCollection):
                        transoff = x.get_offset_transform()
                        x._transOffset = r_box+transoff     
                ax.set_xlim(0, 2*np.pi + 3*np.pi/5)
                ax.set_ylim(0, 2*np.pi + 3*np.pi/5)
                ax.set_aspect('equal', 'box') 
                ax.axis('off')   
        plt.show()


In [ ]:
############### Ratemaps ###############
for mouse_sess in ind_fis:
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
    for i, fi in enumerate(files):
        files[i] = fi.replace('\\', '/')

    ind_fi = ind_fis[mouse_sess]
    f = np.load(data_dir + '/' + mouse_sess + '_corrs.npz', allow_pickle = True)
    acorrs = f['acorr'][()]
    f.close()
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    f.close()
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]
    for ii, fis in ind_fi:
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        sspikes_of1_fis = {}
        pcorrmed = 0
        pcorr = []
        f = np.load('Results/' + mouse_sess + '/' + mod_name + '_coords_aligned.npz' , allow_pickle = True) 
        coords_aligned = f['coords_aligned'][()]
        f.close()
        
        for fi in fis:
            fi = files[fi]
            fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
            f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
            sspikes_of1_fis[fi] = f['sspikes_of1']
            pcorr1 = f['pcorr1']
            pcorr2 = f['pcorr2']
            f.close()
            if np.median(pcorr1)> pcorrmed:
                pcorrmed = np.median(pcorr1)
                pcorr = pcorr1
            if np.median(pcorr2)> pcorrmed:
                pcorrmed = np.median(pcorr2)
                pcorr = pcorr1            
                         

        num_neurons = len(pcorr)
        numfigs = len(fis) + 1
        numw = 5
        numh = int(np.ceil(num_neurons/numw))
        outer1 = gridspec.GridSpec(1, numw)
        fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 120)
        plt.viridis()
        nw = 0
        sig1 = 1
        numbins1 = 30
                        
        torsort = np.flip(np.argsort(pcorr))
        fi_dark = ''
        for fi in files:
            if fi.lower().find('dark')>-1:
                fi_dark = fi
                break
        if len(fi_dark) == 0:
            fi_dark = files[0]
                        
        ac = acorrs[fi_dark][indskeep_bu[ind1==ii],:].copy()

        for nn, n in enumerate(torsort):
            nnn = nn%numh
            if nnn == 0:
                outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .3)
                gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.2,wspace = .0)
                nw += 1
            xnum = 0

            ax = plt.subplot(gs2[nnn,xnum]) 
            xnum += 1
            ax.bar(np.arange(300),ac[n,:300], width = 1, color = 'k')
            ax.set_xlim([0,300])
            ax.set_ylim([0, 0.7])
            ax.set_xticks([0, 100, 200, 300])
            ax.set_yticks([0.0, 0.2, 0.4, 0.6])
            ax.set_yticklabels('')
            ax.set_xticklabels('')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.set_aspect(1/ax.get_data_ratio())
            ax.set_xticks([])
            ax.set_yticks([])

            for fi in fis:
                fi = files[fi]
                ax = plt.subplot(gs2[nnn,xnum]) 
                xnum += 1    
                mtot_tmp, __, __, circ  = binned_statistic_2d(coords_aligned[fi][:,0],
                                                          coords_aligned[fi][:,1],
                                                          sspikes_of1_fis[fi][:,n], 
                                                          statistic = 'mean', 
                                                          bins = 30,
                                                          expand_binnumbers = True)    
                nans = np.isnan(mtot_tmp)
                mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])                
                mtot_tmp = smooth_tuning_map(mtot_tmp, numbins1+1, sig1, bClose = False) 
                mtot_tmp[nans] = -np.inf
                maxtot = np.sort(mtot_tmp.flatten())
                mintot = maxtot[int(0.025*len(maxtot))]
                maxtot = maxtot[int(0.975*len(maxtot))]
                        
                ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], vmin = 0, vmax = maxtot)
                ax.set_xticks([])
                ax.set_yticks([])
                ax.set_aspect(1/ax.get_data_ratio())
                r_box = transforms.Affine2D().skew_deg(15,15)
                for x in ax.images + ax.lines + ax.collections:
                    trans = x.get_transform()
                    x.set_transform(r_box+trans) 
                    if isinstance(x, PathCollection):
                        transoff = x.get_offset_transform()
                        x._transOffset = r_box+transoff     
                ax.set_xlim(0, 2*np.pi + 3*np.pi/5)
                ax.set_ylim(0, 2*np.pi + 3*np.pi/5)
                ax.set_aspect('equal', 'box') 
                ax.axis('off')   
        plt.show()

## Plot phase distribution

In [ ]:
num_rand = 0

for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
    print(mouse_sess)
    f = np.load(data_dir + '/' + mouse_sess + '_corrs.npz', allow_pickle = True)
    acorrs = f['acorr'][()]
    f.close()
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    f.close()
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
    for ii , fi in enumerate(files):
        files[ii] = fi.replace('\\', '/')
    
                  
    for ii, fis in ind_fi:
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        pcorrmed = 0
        pcorr = []
        f = np.load('Results/' + mouse_sess + '/' + mod_name + '_coords_aligned.npz' , allow_pickle = True) 
        coords_aligned = f['coords_aligned'][()]
        f.close()
        phases_1 = {}
        phases_rand = {}
        for fi in fis:
            fi = files[fi]
            print(fi)

            fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
            f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
            sspikes_of1 = f['sspikes_of1']            
            movetimes0 = f['movetimes0']
            f.close()
                        
            ##########    Get phases   ##########
            coords_bin, inds_label =  get_coord_distribution_binned(coords_aligned[fi][movetimes0], 
                                                                    numbins = 50, overlap = 0.)        
            phases_1[fi] = get_phases_binned(sspikes_of1[movetimes0], coords_bin, inds_label)  

            ##########    Get phases shuffled ##########
            num_times_all = len(coords_aligned[fi])
            num_neurons = len(sspikes_of1[0,:])
            phases_rand[fi] = np.zeros((num_rand, num_neurons, 2))
            for rit in range(num_rand):
                spk_rand = np.zeros_like(sspikes_of1)
                for n in range(num_neurons):
                    np.random.seed(rit*num_neurons + n)
                    randnum = np.random.randint(num_times_all)
                    spk_rand[:, n] = np.roll(sspikes_of1[:, n], randnum, axis = 0)                
                phases_rand[fi][rit, :, :] = get_phases_binned(spk_rand, coords_bin, inds_label) 

            plot_phase_distribution(phases_1[fi], phases_1[fi])
        np.savez('Results/' + mouse_sess + '/' + mod_name + '_phases',
                 phases_1 = phases_1, 
                 phases_rand = phases_rand
                )
        for it1, fi1 in enumerate(fis[:-1]):
            fi1 = files[fi1]
            for it2, fi2 in enumerate(fis[it1+1:]):
                fi2 = files[fi2]
                plot_phase_distribution(phases_1[fi1], phases_1[fi2])

In [ ]:
phase_match_all = {}
phase_match_rand_all = {}
for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
    print(mouse_sess)
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    f.close()
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
    for ii , fi in enumerate(files):
        files[ii] = fi.replace('\\', '/')
    
                  
    for ii, fis in ind_fi:
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        print(mod_name)
        f = np.load('Results/' + mouse_sess + '/' + mod_name + '_phases.npz',allow_pickle = True)
        phases_1 = f['phases_1'][()]
        f.close()

        num_rand = 1000
        for it1, fi1 in enumerate(fis[:1]):
            fi1 = files[fi1]
            for it2, fi2 in enumerate(fis[1:]):
                fi2 = files[fi2]
                print(fi1,fi2)
                num_neurons = len(phases_1[fi1]) 
                dist_rand = np.zeros((num_rand, num_neurons))
                dist =  np.sqrt(np.sum(np.square(np.arctan2(np.sin(phases_1[fi1] - phases_1[fi2]),
                                              np.cos(phases_1[fi1] - phases_1[fi2]))),1))
                for i in range(num_rand):
                    np.random.seed(i)
                    inds = np.arange(num_neurons)
                    np.random.shuffle(inds)
                    dist_rand[i,:] =  np.sqrt(np.sum(np.square(np.arctan2(np.sin(phases_1[fi1] - phases_1[fi2][inds,:]),
                            np.cos(phases_1[fi1] - phases_1[fi2][inds,:]))),1))
                plot_cumulative_stat(dist, dist_rand, [0,np.sqrt(2)*np.pi],1, [0,np.sqrt(2)*np.pi], [0,0.5, 1], [0,np.sqrt(2)*np.pi], [0,1.1])
                plt.show()
                phase_match_all[fi1+'_' + fi2 + mod_name] = dist
                phase_match_rand_all[fi1+'_' + fi2 + mod_name] = dist_rand

In [ ]:
num_mods = len(phase_match_all)
bSems = False
cs = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

fig = plt.figure(figsize=(2,6), dpi = 120)
ax = fig.add_subplot(111)
xs = [0.01, 0.09]

mod_nums = {}
count = 0
for i in phase_match_all:
    mouse_sess = i[i.find('np'):i.find('np')+9] 
    mod_name = i[i.find('mod'):]
    print(mouse_sess, mod_name)
    if mouse_sess + mod_name not in list(mod_nums.keys()):
        mod_nums[mouse_sess + mod_name] = count
        count += 1 

for i in phase_match_all:
    mouse_sess = i[i.find('np'):i.find('np')+9] 
    mod_name = i[i.find('mod'):]
    currcol = cs[mod_nums[mouse_sess + mod_name]]
    p1 = np.mean(phase_match_all[i])
    p11 = np.std(phase_match_all[i])/np.sqrt(len(phase_match_all[i]))
    p2 = np.mean(np.mean(phase_match_rand_all[i],1))
    p22 = np.std(phase_match_rand_all[i])/np.sqrt(len(phase_match_rand_all[i].flatten()))
    ax.scatter(xs, [p1, p2], 
               s = 150, marker='.', zorder=-1, 
               alpha  = 0.5, c = currcol)
    ax.plot(xs, [p1, p2], 
            lw = 1, alpha = 0.5, zorder = -2, c = currcol)
    ax.plot([xs[0],xs[0]],[p1-p11, p1+p11],  
            lw = 1, alpha = 0.5, zorder = -2, c = currcol)
    ax.plot([xs[1],xs[1]],[p2-p22, p2+p22],  
            lw = 1, alpha = 0.5, zorder = -2, c = currcol)

ax.set_xlim([0,0.1])
ax.set_ylim([0.,150/360*2*np.pi])

ax.set_xticks(xs)
ys = [0, 75/360*2*np.pi, 150/360*2*np.pi]
ax.set_xticklabels(np.zeros(len(xs),dtype=str))
ax.xaxis.set_tick_params(width=1, length =7)
ax.set_yticks(ys)
ax.set_yticklabels(np.zeros(len(ys),dtype=str))
ax.yaxis.set_tick_params(width=1, length =7)


x0,x1 = ax.get_xlim()
y0,y1 = ax.get_ylim()
ax.set_aspect(abs(x1-x0)/(abs(y1-y0))*3.5)

plt.gca().axes.spines['top'].set_visible(False)
plt.gca().axes.spines['right'].set_visible(False)



## Plot Correlation distance

In [ ]:
for mouse_sess in ind_fis:
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
    ind_fi = ind_fis[mouse_sess]
    print(mouse_sess)
    print(ind_fi)
    f = np.load(data_dir + '/' + mouse_sess + '_corrs.npz', allow_pickle = True) 
    acorr = f['acorr'][()]
    t_acorr = f['t_acorr'][()]
    Xcorr = f['Xcorr'][()]
    f.close()

    num_neurons = len(Xcorr[files[0]])
    f = np.load('Results/' + mouse_sess + '/inds.npz', allow_pickle = True)
    indskeep_bu = f['indskeep_bu'] 
    f.close()
    
    d1 = np.zeros((len(indskeep_bu),len(indskeep_bu)))
    for x in Xcorr:    
        xtmp = Xcorr[x]
        xtmp = xtmp[indskeep_bu,:]
        xtmp = xtmp[:, indskeep_bu]
        dd1 = squareform(pdist(np.square(xtmp), 'correlation'))
        d1 += dd1
    infs = np.isinf(d1)
    d1[infs] = np.max(d1[~infs])*2
    d1 /= len(Xcorr)
    dvals = d1[np.triu_indices(len(d1),1)]
    print('num_neurons', len(d1))
    ind1 = get_ind(d1, 0.46, linkage = 'average', bPlot = True)
    plt.axis('off')
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0] 
    
    print(rel_inds)
    for r in rel_inds:
        print(r, np.where(np.sort(ind1)==r))
    plt.show()


## Plot unwrapped

In [ ]:
coords_aligned_mod1 = {}
mouse_sess = 'npI1_0417'
ind_fi = ind_fis[mouse_sess]
dim = 7

f = np.load('Results/' + mouse_sess + '/inds.npz')
ind1 = f['ind1']
indskeep_bu = f['indskeep_bu']
f.close()
files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
files.sort()

numinds = np.bincount(ind1)
rel_inds = np.where((numinds>=15) & (numinds<=150))[0]


for ii, fis in ind_fi:
    ii = rel_inds[ii]
    mod_name = 'mod' + str(ii)
    sspikes_of1_fis = {}
    pcorrmed = 0
    pcorr = []

    cs =   {}
    t0 = 0
    gains_means = {}
    gains_stds = {}
    gains_curr = {}
    coords_mod = {}
    for fi in fis:
        fi = files[fi]
        fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
        f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
        sspikes_of1_fis = f['sspikes_of1']
        movetimes0_fis = f['movetimes0']
        movetimes1_fis = f['movetimes1']
        indstemp_fis = f['indstemp']
        coords_ds_fis = f['coords_ds']
        f.close()
        
        coords_mod[fi] = get_coords_all(sspikes_of1_fis, 
                                      coords_ds_fis,   
                                      movetimes0_fis, 
                                      indstemp_fis, 
                                      dim = 7, 
                                      bPCA = True,
                                     )
    coords_aligned_mod1[ii] = coords_mod.copy()



In [ ]:
for fi in fis[:]:
    fi = files[fi]
    coords_aligned_mod1[8][fi], combs1, pshift1 = align_coords(coords_mod1 = coords_aligned_mod1[8][fi], 
                                                          coords_mod2 = coords_aligned_mod1[3][fi], bMod = True
                                                            )
    coords_aligned_mod1[10][fi], combs1, pshift1 = align_coords(coords_mod1 = coords_aligned_mod1[10][fi], 
                                                          coords_mod2 = coords_aligned_mod1[3][fi], bMod = True
                                                            )

In [ ]:
cctrials = {}
for ii, fis in ind_fi:     
    ii = rel_inds[ii]
    for fi in fis[:]:
        fi = files[fi]
        coordscurr = coords_aligned_mod1[ii][fi].copy()
        cc = np.arctan2(gaussian_filter1d(np.sin(coordscurr),sigma = 50,axis = 0),
                        gaussian_filter1d(np.cos(coordscurr),sigma = 50,axis = 0))%(2*np.pi)
        cctrials[str(ii) + fi] = unwrap_coords(cc)


In [ ]:
for j in range(1):
    fig, ax1 = plt.subplots(1,1, dpi = 120)
    cols = ['Reds', 'Greens', 'Blues']
    count = 0
    for i, cind in enumerate(cctrials):
        if i % 3 == j:
            ccplot = cctrials[cind].T/(2*np.pi)
            if j == 1:
                ccplot[1,:] = -ccplot[1,:]
            else:
                ccplot = -ccplot
            
            ax1.scatter(*ccplot, 
                        c = np.arange(len(cctrials[cind])), cmap = cols[count])
            count += 1
    xt = np.arange(0,ax1.get_xlim()[1], 100)
    print(xt)
    yt = np.arange(0,ax1.get_ylim()[1], 100)
    ax1.set_xticks(xt)
    ax1.set_yticks(yt)
    ax1.set_xticklabels('')
    ax1.set_yticklabels('')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)            
    ax1.set_aspect(1)
            

In [ ]:
for j in range(1,2):
    fig, ax1 = plt.subplots(1,1, dpi = 120)
    cols = ['Reds', 'Greens', 'Blues']
    count = 0
    for i, cind in enumerate(cctrials):
        if i % 3 == j:
            ccplot = cctrials[cind].T/(2*np.pi)
            if j == 1:
                ccplot[1,:] = -ccplot[1,:]
                ccplot = np.flip(ccplot,0)
            else:
                ccplot = -ccplot
            
            ax1.scatter(*ccplot, 
                        c = np.arange(len(cctrials[cind])), cmap = cols[count])
            count += 1
    xt = np.arange(0,ax1.get_xlim()[1], 100)
    xt = [0,100]
    
    print(xt)
    yt = np.arange(0,ax1.get_ylim()[1], 100)
    ax1.set_xticks(xt)
    ax1.set_yticks(yt)
    ax1.set_xticklabels('')
    ax1.set_yticklabels('')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)            
    ax1.set_aspect(1)
            

In [ ]:
for j in range(2,3):
    fig, ax1 = plt.subplots(1,1, dpi = 120)
    cols = ['Reds', 'Greens', 'Blues']
    count = 0
    for i, cind in enumerate(cctrials):
        if i % 3 == j:
            ccplot = cctrials[cind].T/(2*np.pi)
            if j == 1:
                ccplot[1,:] = -ccplot[1,:]
                ccplot = np.flip(ccplot,0)
            else:
                ccplot = -ccplot
            
            ccplot[0,:] = -ccplot[0,:]
            
            ax1.scatter(*ccplot, 
                        c = np.arange(len(cctrials[cind])), cmap = cols[count])
            count += 1
    xt = np.arange(0,ax1.get_xlim()[1], 100)
    print(xt)
    yt = np.arange(0,ax1.get_ylim()[1], 100)
    ax1.set_xticks(xt)
    ax1.set_yticks(yt)
    ax1.set_xticklabels('')
    ax1.set_yticklabels('')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)            
    ax1.set_aspect(1)
            

## Plot Tori

In [ ]:
pathihnd

In [ ]:

cols = ['Reds', 'Greens', 'Blues']
for i1, (ii, fis) in enumerate(ind_fi):     
    ii = rel_inds[ii]    
    for fi in fis[:1]:
        fi = files[fi]
        fig = plt.figure(dpi = 320)
        cctrial = cctrials[str(rel_inds[ind_fi[0][0]]) + fi].copy()
        cctrial1 = cctrial.copy()

        r1 = 1.5
        r2 = 1.0
        x = (r1 + r2*np.cos(cctrial1[:,0]))*np.cos(cctrial1[:,1]) 
        y = (r1 + r2*np.cos(cctrial1[:,0]))*np.sin(cctrial1[:,1])  
        z =  r2*np.sin(cctrial1[:,0])

        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(x,
                   y,
                   z, edgecolors = None,
                   c = np.arange(len(z)),
                   cmap = cols[i1], 
                   alpha = 0.3,s = 0.05, zorder = -2)
        ax.axis('off')
        ax.view_init(66,180)                
        fig.tight_layout()
        plt.show()

## Plot 2D

In [ ]:
k = 1000          # number of neighbours for downsampling
dt_curr = 0.01    # time bin interval
time_resolution = 100000 

for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
    
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    f.close()
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]
    
    mouse_folder = 'Results/' + mouse_sess
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
    print(files)
    data_trials = {}
    speed = {}    
    posxx = {}
    postt = {}
    postrial = {}
    gain = {}
    contrast = {}
    lickt = {}
    lickx = {}
    min_time = 0
    max_times = {}
    for fi in files[:]:
        data = loadmat(fi)
        max_time = data['sp']['st'].max() +dt_curr
        max_times[fi] = max_time.copy()

        xtmp = data['posx'].copy()
        xtmp += (data['trial']-1)*400

        postt[fi], posxx[fi], __, speed[fi],__ = get_pos(xtmp, [], data['post'], [],
                                                         dt_curr, res = time_resolution, min_time = min_time, max_time = max_time)
        posxx[fi] = posxx[fi]%400

        postrial[fi] = data['trial']
        gain[fi] = data['trial_gain']
        contrast[fi] = data['trial_contrast']
        lickt[fi] = data['lickt']        
        lickx[fi] = data['lickx']
        tt_dig = np.digitize(postt[fi], data['post'])-1    
        postrial[fi] = data['trial'][tt_dig]
        data_trials[fi] = np.unique(data['trial'])


    ############# Plot coordinates vs wheel running #############
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
    cctrials = []                  
    for ii, fis in ind_fi:
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        for fi in fis[:]:
            fi = files[fi]            
            fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
            f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
            sspikes_of1_fis = f['sspikes_of1']
            movetimes0_fis = f['movetimes0']
            movetimes1_fis = f['movetimes1']
            indstemp_fis = f['indstemp']
            coords_ds_fis = f['coords_ds']
            f.close()

            coordscurr = get_coords_all(sspikes_of1_fis, 
                                          coords_ds_fis,   
                                          movetimes0_fis, 
                                          indstemp_fis, 
                                          dim = 7, 
                                          bPCA = True,
                                         )

            
            len_plot = postt[fi][-1]
            times = np.arange(0,np.where(postt[fi]>=len_plot)[0][0])
            dx = np.abs(posxx[fi][1:]-posxx[fi][:-1])[times]
            times = times[dx<10]

            im = plt.imread('image044.png')
            im = np.array(im)
            a1 = np.rot90(im, 1)
            cc = np.arctan2(gaussian_filter1d(np.sin(coordscurr[times,:]),sigma = 50,axis = 0),
                            gaussian_filter1d(np.cos(coordscurr[times,:]),sigma = 50,axis = 0))%(2*np.pi)
            bCos = True
            eps = 1e-10
            digitized = np.concatenate((np.digitize(np.cos(cc[:, 0]), np.linspace(-1-eps,1+eps, len(a1)+1))[:,np.newaxis], 
                                        np.digitize(np.cos(cc[:, 1]), np.linspace(-1-eps,1+eps, len(a1)+1))[:,np.newaxis]),1)
            cc1 = []
            for i in range(len(times)):
                cc1.append(a1[digitized[i,1]-1, digitized[i,0]-1]) 
            cs11 = CubicSpline(times, np.sin(cc))
            cc11 = CubicSpline(times, np.cos(cc))
            fig = plt.figure(figsize = (10,5), dpi = 200)
            plt.axis('off')
            plt.hsv()

            heights = [2,]
            gs0 = gridspec.GridSpec(1, 1, figure=fig, height_ratios=heights, hspace = 0.1)
            ax1 = fig.add_subplot(gs0[0])

            ax1.axis('off')
            plt.xlim([0,len_plot])
            im = ax1.scatter(postt[fi][times], 
                             posxx[fi][times], s = 10, c = cc1)
            plt.show()
            
            if (fi.find('gain')>-1) or (fi.find('contrast')>-1):
                fig = plt.figure(figsize = (10,1), dpi = 100)
                ax1 = fig.add_subplot(111)
                ax1.set_xlim([0,len_plot])    
                ax1.set_ylim([-0.1, 1.1])    
                ax1.plot(postt[fi][times], contrast[fi][postrial[fi]-1][times]/100, c = 'r')
                ax1.plot(postt[fi][times], gain[fi][postrial[fi]-1][times], c = 'g')
                ax1.set_xticklabels('')
                ax1.set_yticklabels('')
                ax1.spines['top'].set_visible(False)
                ax1.spines['right'].set_visible(False)

                plt.show()


## Gain Estimation 

In [ ]:
from scipy.stats import linregress
lin_thr = 0.5
num_reg = 100
sig = 20
cctrial_fi_all = {}
gains_means_all = {}
gains_std_all = {}
gains_all = {}
cc_gains_total = {}

#for mouse_sess in ind_fis:
dt_curr = 0.01
time_resolution = 100000
min_time = 0


In [ ]:
for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
    dim = 7
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    f.close()
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()

    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]

    for ii, fis in ind_fi:
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        cs =   {}
        t0 = 0
        gains_means = {}
        gains_stds = {}
        gains_curr = {}
        for fi in fis:
            fi = files[fi]
            if fi.find('gain')>-1:
                fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
                f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
                sspikes_of1_fis = f['sspikes_of1']
                movetimes0_fis = f['movetimes0']
                movetimes1_fis = f['movetimes1']
                indstemp_fis = f['indstemp']
                coords_ds_fis = f['coords_ds']
                f.close()

                cctrial_temp = get_coords_all(sspikes_of1_fis, 
                                              coords_ds_fis,   
                                              movetimes0_fis, 
                                              indstemp_fis, 
                                              dim = 7, 
                                              bPCA = True,
                                             )


                if sig>0:
                    cctrial_temp = np.arctan2(gaussian_filter1d(np.sin(cctrial_temp),
                                                                sigma = sig,axis = 0),
                                              gaussian_filter1d(np.cos(cctrial_temp),
                                                                  sigma = sig,axis = 0))%(2*np.pi)
                cctrial = unwrap_coords(cctrial_temp)
                data = loadmat(fi)
                gain = data['trial_gain']
                contrast = data['trial_contrast']  
                dt_curr = 0.01
                max_time = data['sp']['st'].max() +dt_curr
                max_times = max_time.copy()
                xtmp = data['posx'].copy()
                xtmp += (data['trial']-1)*400
                postt, posxx, __, speed,__ = get_pos(xtmp, [], data['post'], [],
                                                                 dt_curr, res = time_resolution, min_time = min_time, max_time = max_time)

                tt_dig = np.digitize(postt, data['post'])-1    
                postrial = data['trial'][tt_dig]
                data_trials = np.unique(data['trial'])
                for contrast_temp in np.unique(contrast):
                    traj_keep = (contrast == contrast_temp)        
                    trial_range = np.unique(postrial)[traj_keep]
                    gain_trials = gain[traj_keep]  
                    if len(np.unique(gain_trials))>1:
                        gains = np.unique(gain_trials)
                        trials_all = postrial.copy()
                        gg = []
                        cc_gains_all = []
                        for i, g1 in enumerate(gains):
                            trial_curr = trial_range[gain_trials==g1]
                            cc_gains = []
                            for j, cctrial1 in enumerate(trial_curr):       
                                cccurr = cctrial[trials_all == cctrial1]
                                cccurr = cccurr[np.linspace(0,len(cccurr)-1, num_reg).astype(int)]
                                lin1 = linregress(np.arange(len(cccurr)),cccurr[:,0])
                                lin2 = linregress(np.arange(len(cccurr)),cccurr[:,1])
                                cccurr = np.concatenate(((lin1[0]*np.arange(len(cccurr))+lin1[2])[:,np.newaxis],
                                                        (lin2[0]*np.arange(len(cccurr))+lin2[2])[:,np.newaxis]),1)
                                if lin_thr>0:
                                    if (np.abs(lin1[2])>lin_thr) & (np.abs(lin2[2])>lin_thr):
                                        maxx,maxy = cccurr[-1,:]
                                        minx,miny = cccurr[0,:]
                                        cc_gains.extend([np.sqrt(np.sum(np.square([maxx-minx, maxy-miny])))])
                                else:
                                    maxx,maxy = cccurr[-1,:]
                                    minx,miny = cccurr[0,:]
                                    cc_gains.extend([np.sqrt(np.sum(np.square([maxx-minx, maxy-miny])))])

                            if len(cc_gains) >2:
                                cc_gains_all.append(cc_gains)
                                gg.extend([g1])
                        gains_mean = np.zeros(len(gg))
                        gains_std = np.zeros((len(gg), 2))
                        cc_gains_total[mouse_sess + fi_name + mod_name + '_c' + str(contrast_temp)] = cc_gains_all.copy()
                        for i in range(len(gg)):
                            cc_gains_all[i] /= np.median(cc_gains_all[-1])
                            gains_mean[i] = np.median(cc_gains_all[i]) 
                            #gains_std[i] = [gains_mean[i]-np.std(cc_gains_all[i]), 
                            #                gains_mean[i]+np.std(cc_gains_all[i])]
                            gains_std[i] =[np.quantile(cc_gains_all[i], 0.25), np.quantile(cc_gains_all[i], 0.75)]
                        gains_means[fi + str(contrast_temp)] = gains_mean.copy()
                        gains_stds[fi + str(contrast_temp)] = gains_std.copy()
                        gains_curr[fi + str(contrast_temp)] = np.array(gg).copy()

                        fig1 = plt.figure()
                        ax1 = fig1.add_subplot(111)
                        ax1.scatter(gg, gains_mean, marker = 'o', s = 50, lw = 0.1, c= 'k')
                        ax1.plot(gg, gains_mean, ls = ':', lw = 1.5, c= 'k')
                        for i in range(len(gg)):
                            ax1.plot([gg[i],gg[i]], [gains_std[i][0], gains_std[i][1]], 
                                     lw = 0.75,c = 'k')
                        ax1.set_xticks(gg)  
                        ax1.set_ylim([0.5,2.4])
                        ax1.set_xlim([0.45,1.05])
                        ax1.plot(plt.gca().get_xlim(), np.ones(2), c = 'k', ls = '--')
                        fig1.tight_layout()
                        print(gains_mean)
                        plt.show()         

                        gains_means_all[mouse_sess + mod_name] = gains_means.copy()
                        gains_std_all[mouse_sess + mod_name] = gains_stds.copy()
                        gains_all[mouse_sess + mod_name] = gains_curr.copy()


In [ ]:
from scipy.stats import kruskal

pmax = -np.inf 
Zmin = np.inf 
pmin = np.inf 
Zmax = -np.inf 
for cgt in cc_gains_total:
    for i in range(len(cc_gains_total[cgt])):
        print(cgt, len(cc_gains_total[cgt][i]))
        k = kruskal(cc_gains_total[cgt][i], cc_gains_total[cgt][-1])
        print(k)
        if (i == 0):# & (< 0):
            pmax= max(pmax, k[1])
            Zmin = min(Zmin, k[0])
            pm= max(pmax, k[1])
            Zmin = min(Zmin, k[0])
    print('')
    print('')
print(pmax, Zmin)

## Videos

In [ ]:
from matplotlib import animation, cm, transforms, pyplot as plt
from IPython.display import HTML
from matplotlib.collections import PathCollection


In [ ]:
for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
       if sig>0:
                cctrial_temp = np.arctan2(gaussian_filter1d(np.sin(cctrial_temp),
                                                            sigma = sig,axis = 0),
                                          gaussian_filter1d(np.cos(cctrial_temp),
                                                              sigma = sig,axis = 0))%(2*np.pi)
            

In [ ]:
time_resolution = 100000
dt_curr = 0.01
min_time = 0

mouse_sess = 'npI1_0417'

dim = 7
f = np.load('Results/' + mouse_sess + '/inds.npz')
ind1 = f['ind1']
indskeep_bu = f['indskeep_bu']
f.close()
files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
files.sort()

numinds = np.bincount(ind1)
rel_inds = np.where((numinds>=19) & (numinds<=150))[0]

for ii, fis in ind_fi:
    ii = rel_inds[ii]
    mod_name = 'mod' + str(ii)
    cs =   {}
    t0 = 0
    gains_means = {}
    gains_stds = {}
    gains_curr = {}
    for fi in fis:
        fi = files[fi]
        fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
        f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
        sspikes_of1_fis = f['sspikes_of1']
        movetimes0_fis = f['movetimes0']
        movetimes1_fis = f['movetimes1']
        indstemp_fis = f['indstemp']
        coords_ds_fis = f['coords_ds']
        f.close()

        cctrial_temp = get_coords_all(sspikes_of1_fis, 
                                      coords_ds_fis,   
                                      movetimes0_fis, 
                                      indstemp_fis, 
                                      dim = 7, 
                                      bPCA = True,
                                     )


#            cctrial = unwrap_coords(cctrial_temp)
        data = loadmat(fi)
        gain = data['trial_gain']
        contrast = data['trial_contrast']  
        dt_curr = 0.01
        max_time = data['sp']['st'].max() +dt_curr
        max_times = max_time.copy()
        xtmp = data['posx'].copy()
        xtmp += (data['trial']-1)*400
        time_resolution = 100000
        dt_curr = 0.01
        min_time = 0
        postt, posxx, __, speed,__ = get_pos(xtmp, [], data['post'], [],
                                                         dt_curr, res = time_resolution, min_time = min_time, max_time = max_time)

        tt_dig = np.digitize(postt, data['post'])-1    
        postrial = data['trial'][tt_dig]
        data_trials = np.unique(data['trial'])

postt, posxx, __, speed,__ = get_pos(xtmp, [], data['post'], [],
                                                 dt_curr, res = time_resolution, min_time = min_time, max_time = max_time)

tt_dig = np.digitize(postt, data['post'])-1    
postrial = data['trial'][tt_dig]
data_trials = np.unique(data['trial'])


In [ ]:
coords_ds_fis[1,:] = 1-coords_ds_fis[1,:]
starttime =0
num_frames = 8000
#times = np.arange(0,len(sspk1),5)

coordsnew_mod11,coordsbox1 = get_coords_distribution1(sspikes_of1_fis[movetimes0_fis,:], 
                                                      sspikes_of1_fis, coords_ds_fis,
                                                      indstemp_fis, 
                                                      starttime = starttime, num_frames = num_frames)
#                                       gaussian_filter1d(sspk1,sigma = 5, axis = 0),#[times,:], 
                                       

In [ ]:
######### Video without gain #########

headpos1 = preprocessing.minmax_scale(posxx%400)
ys = [0, 400]
coordsbox11 = (coordsbox1)*2*np.pi/3+2*np.pi/3

ttmp = 2000

tt1 = postt.copy()
tt1[:starttime] = tt1[starttime]
plt.viridis()

fig = plt.figure(figsize = (8,5), dpi = 160)#,constrained_layout=True)
gs = fig.add_gridspec(2, 2, height_ratios = [3,1],width_ratios = [1,1.5])

ax = fig.add_subplot(gs[0, 0])
img, = ax.plot(tt1[:ttmp],headpos1[:ttmp], c = 'k')
minh, maxh = 0, 1
ax.set_ylim(minh, maxh)
ax.set_xlim( tt1[0], tt1[ttmp])
plt.gca().axes.spines['top'].set_visible(False)
plt.gca().axes.spines['right'].set_visible(False)


ax2 = fig.add_subplot(gs[:, 1])
ax2.set_xlim( np.sin(np.pi/12)*2*np.pi, np.cos(np.pi/12)*2*np.pi)
ax2.set_ylim( np.sin(np.pi/12)*2*np.pi, np.cos(np.pi/12)*2*np.pi)
ax2.set_aspect('equal','box')
ax2.axis('off')
cc_comb = np.concatenate((coordsnew_mod11[0,:,:],coordsnew_mod11[0,:,:],coordsnew_mod11[0,:,:]),1)
cc_comb = np.concatenate((cc_comb,cc_comb,cc_comb),0)

r_box = transforms.Affine2D().skew_deg(15,15)


img2 = ax2.imshow(cc_comb, origin = 'lower',
                  extent = [0,2*np.pi, 0, 2*np.pi], vmin = 0, vmax = 10)
img3 = ax2.scatter([],[], c = [[0.7,0.7,0.7]], s = 5, alpha = 0.6, transform=r_box + ax2.transData)
img4 = ax2.scatter([],[], c = [[0.9, 0.2,0.2]], s = 15, alpha = 1, transform=r_box + ax2.transData)

ax2.plot([2/3*np.pi,2/3*np.pi],[2/3*np.pi,4/3*np.pi], c = [0.5,0.5,0.5], ls = '--')
ax2.plot([2/3*np.pi, 4/3*np.pi],[2/3*np.pi,2/3*np.pi], c = [0.5,0.5,0.5], ls = '--')

ax2.plot([4/3*np.pi,4/3*np.pi],[2/3*np.pi,4/3*np.pi], c = [0.5,0.5,0.5], ls = '--')
ax2.plot([2/3*np.pi, 4/3*np.pi],[4/3*np.pi,4/3*np.pi], c = [0.5,0.5,0.5], ls = '--')


im = [img,  img2,img3, img4, ]
def init():
    im[0].set_data([],[])
    
    im[1].set_data([[]])
    im[2].set_offsets(np.array([[],[]]).T)
    im[3].set_offsets([[]])  
    
    return im

for x in (ax2.images + ax2.lines + ax2.collections + ax2.texts):
    trans = x.get_transform()
    x.set_transform(r_box+trans) 
    if isinstance(x, PathCollection):
        transoff = x.get_offset_transform()
        x._transOffset = r_box+transoff     

fps = 25
height, width = 1600, 1600
ax2.set_title('Torus')
ax.text(tt1[starttime]-ttmp*0.0023, maxh/2, 'Wheel')

def animate(i):
    i+= starttime    
    if i-starttime < ttmp:
        im[0].set_data(tt1[starttime:i+1], headpos1[starttime:i+1])
        im[0].axes.axis([tt1[starttime], tt1[starttime+ttmp], minh, maxh])
        xs = [tt1[starttime], tt1[starttime+ttmp]]
    else:    
        im[0].set_data(tt1[i-ttmp:i+1], headpos1[i-ttmp:i+1])
        im[0].axes.axis([tt1[i-ttmp], tt1[i+1], minh, maxh])
        xs = [tt1[i-ttmp], tt1[i+1]]            
    ax.set_xticks(xs)
    ax.set_xticklabels([str(int(xi)) + 's' for xi in xs])
    ax.set_yticks([minh, maxh])
    ax.set_yticklabels([str(yi) + 'cm' for yi in ys])        
    cc_comb = np.concatenate((coordsnew_mod11[i-starttime,:,:],
                              coordsnew_mod11[i-starttime,:,:],
                              coordsnew_mod11[i-starttime,:,:]),1)
    cc_comb = np.concatenate((cc_comb,cc_comb,cc_comb),0)

    im[1].set_data(np.rot90(cc_comb,0))    
    im[2].set_offsets(coordsbox11[starttime:i+1,:])    
    im[3].set_offsets(coordsbox11[i:i+1,:])    
    
    return im
anim = animation.FuncAnimation(fig, animate,# init_func=init,
                               frames=1000, interval=10, blit=True)
#anim = animation.FuncAnimation(fig, animate, init_func=init,
#                               frames=10, interval=50, blit=True)


In [ ]:
anim.save('Results/' + mouse_sess + '/' + files[0].replace(data_dir + '/', '').replace('.mat', '.mp4'))

## Acorr mean 

In [ ]:
cs = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
    dim = 7
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    f.close()
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
#    for ii , fi in enumerate(files):
#        files[ii] = fi.replace('\\', '/')
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]
    f = np.load(data_dir + '/' + mouse_sess + '_corrs.npz', allow_pickle = True) 
    acorr = f['acorr'][()]
    t_acorr = f['t_acorr'][()]
    Xcorr = f['Xcorr'][()]
    f.close()
    
    dt_curr = 0.01
    time_resolution = 100000
    data_trials = {}
    speed = {}    
    posxx = {}
    postt = {}
    postrial = {}
    gain = {}
    contrast = {}
    lickt = {}
    lickx = {}
    min_time = 0
    max_times = {}
    for fi in files[:]:
        data = loadmat(fi)
        max_time = data['sp']['st'].max() +dt_curr
        max_times[fi] = max_time.copy()

        xtmp = data['posx'].copy()
        xtmp += (data['trial']-1)*400

        postt[fi], posxx[fi], __, speed[fi],__ = get_pos(xtmp, [], data['post'], [],
                                                         dt_curr, res = time_resolution, 
                                                         min_time = min_time, max_time = max_time)
        posxx[fi] = posxx[fi]%400

        postrial[fi] = data['trial']
        gain[fi] = data['trial_gain']
        contrast[fi] = data['trial_contrast']
        lickt[fi] = data['lickt']        
        lickx[fi] = data['lickx']
        tt_dig = np.digitize(postt[fi], data['post'])-1    
        postrial[fi] = data['trial'][tt_dig]
        data_trials[fi] = np.unique(data['trial'])

    for ii, fis in ind_fi:
        
        ii = rel_inds[ii]
        
        
        mod_name = 'mod' + str(ii)
        sspikes_of1_fis = {}
        pcorrmed = 0
        pcorr = []
        spk_sim_sqr = {}
        spk_sim_hex = {}
        movetimes0_fis = {}
        movetimes1_fis = {}
        indstemp_fis = {}
        coords_ds_fis = {}

        acorr_real = {}
        for fi in fis:
            fi = files[fi]
            fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
            f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
            sspikes_of1_fis[fi] = f['sspikes_of1']
            pcorr1 = f['pcorr1']
            pcorr2 = f['pcorr2']
            movetimes0_fis[fi] = f['movetimes0']
            movetimes1_fis[fi] = f['movetimes1']
            indstemp_fis[fi] = f['indstemp']
            coords_ds_fis[fi] = f['coords_ds']
            f.close()
            
            num_neurons = len(Xcorr[files[0]])
            f = np.load('Results/' + mouse_sess + '/inds.npz', allow_pickle = True)
            indskeep_bu = f['indskeep_bu'] 
            f.close()

            numbins = 10
                        
            if np.median(pcorr1)>np.median(pcorr2):
                theta_fis[fi] = True
            else:
                theta_fis[fi] = False
            coords_curr = get_coords_all(sspikes_of1_fis[fi], 
                                          coords_ds_fis[fi],   
                                          movetimes0_fis[fi], 
                                          indstemp_fis[fi], 
                                          dim = 7, 
                                          bPCA = True,
                                         )
        
            if theta_fis[fi]:
                coords_curr[:,0] = 2*np.pi- coords_curr[:,0]
            
            coords_bin, inds_label =  get_coord_distribution_binned(coords_curr, 
                                                                    numbins = 50, 
                                                                    overlap = np.sqrt(2)-1)        
            phases_1 = get_phases_binned(sspikes_of1_fis[fi], coords_bin, inds_label)  

            spk_sim_hex[fi] = simulate_spk_hex(coords_curr, phases_1, t = 0.1, nums = 1)
        

        acorr_real = get_acorr(mouse_sess, good_cells = np.arange(num_neurons), data_trials = data_trials,
                          sspk1 = sspikes_of1_fis, files = np.array(files)[[fis]][0], posxx = posxx, postrial = postrial)
        acorr_hex = get_acorr(mouse_sess, good_cells = np.arange(num_neurons), data_trials = data_trials,
                          sspk1 = spk_sim_hex, files = np.array(files)[[fis]][0], posxx = posxx, postrial = postrial)
#        acorr_sqr = get_acorr(mouse_sess, good_cells = np.arange(num_neurons), data_trials = data_trials,
#                          sspk1 = spk_sim_sqr, files = np.array(files)[[fis]], posxx = posxx, postrial = postrial)

        acorr_corr = {}
        for fi in fis:
            fi = files[fi]
            num_neurons = len(spk_sim_hex[fi][0,:])
            acorr_corr[fi] = np.zeros((num_neurons,2))
            for i in range(num_neurons):
                acorr_corr[fi][i,0] = pearsonr(acorr_real[fi][i,1:], acorr_hex[fi][i,1:])[0] 
#                acorr_corr[fi][i,1] = pearsonr(acorr_real[fi][i,1:], acorr_sqr[fi][i,1:])[0] 

            print(np.mean(acorr_corr[fi],0), np.std(acorr_corr[fi],0)/np.sqrt(len(acorr_corr[fi])))

            fig, ax = plt.subplots(1,1)        
            acorr_mean_real = acorr_real[fi] - acorr_real[fi].mean(1)[:,np.newaxis]
            acorrmean = acorr_mean_real.mean(0)
            acorrstd = 1*acorr_real[fi].std(0)/np.sqrt(len(acorr_real[fi][:,0]))
            ax.plot(acorrmean, lw = 2, c= cs[0])
            ax.fill_between(np.arange(len(acorrmean)),acorrmean, acorrmean + acorrstd,
                            lw = 0, color= cs[0], alpha = 0.3)
            ax.fill_between(np.arange(len(acorrmean)),acorrmean, acorrmean - acorrstd,
                            lw = 0, color= cs[0], alpha = 0.3)        

            acorr_mean_hex = acorr_hex[fi] - acorr_hex[fi].mean(1)[:,np.newaxis]
            acorrmean = acorr_mean_hex.mean(0)
            acorrstd = 1*acorr_hex[fi].std(0)/np.sqrt(len(acorr_real[fi][:,0]))
            ax.plot(acorrmean, lw = 2, c= cs[1])
            ax.fill_between(np.arange(len(acorrmean)),acorrmean, acorrmean + acorrstd,
                            lw = 0, color= cs[1], alpha = 0.3)
            ax.fill_between(np.arange(len(acorrmean)),acorrmean, acorrmean - acorrstd,
                            lw = 0, color= cs[1], alpha = 0.3)        

#            acorr_mean_sqr = acorr_sqr[fi] - acorr_sqr[fi].mean(1)[:,np.newaxis]
#            acorrmean = acorr_mean_sqr.mean(0)
#            acorrstd = 1*acorr_sqr[fi].std(0)/np.sqrt(len(acorr_sqr[fi][:,0]))
#            ax.plot(acorrmean, lw = 2, c= cs[3])
#            ax.fill_between(np.arange(len(acorrmean)),acorrmean, acorrmean + acorrstd,
#                            lw = 0, color= cs[3], alpha = 0.3)
#            ax.fill_between(np.arange(len(acorrmean)),acorrmean, acorrmean - acorrstd,
#                            lw = 0, color= cs[3], alpha = 0.3)        
#            ax.set_aspect(1/ax.get_data_ratio())

            plt.gca().axes.spines['top'].set_visible(False)
            plt.gca().axes.spines['right'].set_visible(False)
            plt.xlim([0, 300])
            plt.ylim([-0.2, 0.3])


            xs = [0,100,200,300]
            ax.set_xticks(xs)
            ax.set_xticklabels(np.zeros(len(xs),dtype=str))
            ys = [-0.2, 0.0, 0.3]
            ax.set_yticks(ys)
            ax.set_yticklabels(np.zeros(len(ys),dtype=str))
            print(mouse_sess, fi, ii)
            plt.show()
        #            plt.savefig(folder_curr + '/spatial_acorrs_' + finame, bbox_inches='tight', pad_inches=0.1, transparent = True)

In [ ]:
plt.viridis()
for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
    dim = 7
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    f.close()
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
#    for ii , fi in enumerate(files):
#        files[ii] = fi.replace('\\', '/')

    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]

    for ii, fis in ind_fi:
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        coords_aligned = {}
        sspikes_of1_fis = {}
        movetimes0_fis = {}
        movetimes1_fis = {}
        indstemp_fis = {}
        coords_ds_fis = {}
        dgms_fis = {}
        theta_fis = {}
        for fi in fis:
            fi = files[fi]
            fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
            f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
            sspikes_of1_fis[fi] = f['sspikes_of1']
            movetimes0_fis[fi] = f['movetimes0']
            movetimes1_fis[fi] = f['movetimes1']
            indstemp_fis[fi] = f['indstemp']
            coords_ds_fis[fi] = f['coords_ds']
            pcorr1 = f['pcorr1']
            pcorr2 = f['pcorr2']
            f.close()
            if np.median(pcorr1)>np.median(pcorr2):
                theta_fis[fi] = True
            else:
                theta_fis[fi] = False
            coords_aligned[fi] = get_coords_all(sspikes_of1_fis[fi], 
                                          coords_ds_fis[fi],   
                                          movetimes0_fis[fi], 
                                          indstemp_fis[fi], 
                                          dim = 7, 
                                          bPCA = True,
                                         )   

for mouse_sess in ind_fis:
    ind_fi = ind_fis[mouse_sess]
    dim = 7
    f = np.load('Results/' + mouse_sess + '/inds.npz')
    ind1 = f['ind1']
    indskeep_bu = f['indskeep_bu']
    f.close()
    files = glob.glob(data_dir + '/' + mouse_sess + '_*.mat')
    files.sort()
#    for ii , fi in enumerate(files):
#        files[ii] = fi.replace('\\', '/')
    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=19) & (numinds<=150))[0]
    f = np.load(data_dir + '/' + mouse_sess + '_corrs.npz', allow_pickle = True) 
    acorr = f['acorr'][()]
    t_acorr = f['t_acorr'][()]
    Xcorr = f['Xcorr'][()]
    f.close()
    
    dt_curr = 0.01
    time_resolution = 100000
    for ii, fis in ind_fi:        
        ii = rel_inds[ii]
        mod_name = 'mod' + str(ii)
        
        sspikes_of1_fis = {}
        movetimes0_fis = {}
        movetimes1_fis = {}
        indstemp_fis = {}
        coords_ds_fis = {}
        dgms_fis = {}
        theta_fis = {}
        for fi in fis:
            fi = files[fi]
            print(fi)
            fi_name = fi.replace(data_dir + '/', '').replace('.mat', '').replace(mouse_sess, '')
            f = np.load('Results/' + mouse_sess + '/' + mod_name + '_' + fi_name + '.npz')
            sspikes_of1_fis[fi] = f['sspikes_of1']
            movetimes0_fis[fi] = f['movetimes0']
            movetimes1_fis[fi] = f['movetimes1']
            indstemp_fis[fi] = f['indstemp']
            coords_ds_fis[fi] = f['coords_ds']
            pcorr1 = f['pcorr1']
            pcorr2 = f['pcorr2']
            f.close()
            if np.median(pcorr1)>np.median(pcorr2):
                theta_fis[fi] = True
            else:
                theta_fis[fi] = False
            coords_curr = get_coords_all(sspikes_of1_fis[fi], 
                                          coords_ds_fis[fi],   
                                          movetimes0_fis[fi], 
                                          indstemp_fis[fi], 
                                          dim = 7, 
                                          bPCA = True,
                                         )
        
            if theta_fis[fi]:
                coords_curr[:,0] = 2*np.pi- coords_curr[:,0]
                

            coords_bin, inds_label =  get_coord_distribution_binned(coords_curr, 
                                                                    numbins = 50, 
                                                                    overlap = np.sqrt(2)-1)        
            
            phases_1 = get_phases_binned(sspikes_of1_fis[fi], coords_bin, inds_label)  
            plot_centered_ratemaps(coords_curr, sspikes_of1_fis[fi], phases_1, 50)
            plt.show()